---
title: "Julia Basics"
author: "Pablo Winant"
format:
    html: default
    ipynb: default
---

### What is Julia

- developped at MIT on top of __opensource__ technologies
    - linux / git / llvm
- syntax inspired by Matlab but:
    - more consistent
    - lots of features from high level languages
- everything is JIT-compiled
    - interpreted vs compiled treadeoff
    - -> very fast
    - most of the base library is written in Julia
- opensource/free + vibrant community


Some useful links from QuantEcon:

* [Julia cheatsheet](https://cheatsheets.quantecon.org/julia-cheatsheet.html)
* [Julia-Matlab comparison](https://cheatsheets.quantecon.org/index.html)
* [Julia essentials](https://lectures.quantecon.org/jl/julia_essentials.html)
* [Vectors, arrays and matrices](https://lectures.quantecon.org/jl/julia_arrays.html)

Excellent resources at: [julialang](https://julialang.org/learning/)
- checkout JuliaAcademy, it's free
- ongoing [MOOC](https://computationalthinking.mit.edu/Spring21/) at MIT

### an example of what you shouldn't do in Matlab

How I learnt: interpreted code is slow, so vectorize your coe.

In [2]:
function stupid_loop(I,J,K)
    t = 0.0
    for i=1:I
        for j=1:J
            for k = 1:K
                t += 1.0
            end        
        end
    end
    return t
end
@time [ stupid_loop(1000,1000,i) for i =1:10]

  0.069282 seconds (43.84 k allocations: 2.184 MiB, 8.48% gc time, 57.28% compilation time)


10-element Vector{Float64}:
 1.0e6
 2.0e6
 3.0e6
 4.0e6
 5.0e6
 6.0e6
 7.0e6
 8.0e6
 9.0e6
 1.0e7

In [3]:
@time stupid_loop(1000,1000,1000)

  0.529853 seconds


1.0e9

Code is translated to LLVM code then to instructions for the processor. Note that processor instructions are shorter than LLVM code.

In [3]:
@code_llvm stupid_loop(10,10,10)

;  @ /home/pablo/Teaching/ensae/mie37/tutorials/1_Julia_Basics.ipynb:1 within `stupid_loop`
define double @julia_stupid_loop_1248(i64 signext %0

, i64 signext %1, i64 signext %2) #0 {
top:
;  @ /home/pablo/Teaching/ensae/mie37/tutorials/1_Julia_Basics.ipynb:3 within `stupid_loop`
; ┌ @ range.jl:5 within `Colon`
; │┌ @ range.jl:397 within `UnitRange`
; ││┌ @ range.jl:404 within `unitrange_last`
     %.inv = icmp sgt i64 %0, 0
     %. = select i1 %.inv, i64 %0, i64 0
; └└└
  br i1 %.inv, label %L16.preheader, label %L85

L16.preheader:                                    ; preds = %top
  %.inv26 = icmp sgt i64 %1, 0
  %.24 = select i1 %.inv26, i64 %1, i64 0
  %.inv27 = icmp sgt i64 %2, 0
  %.25 = select i1 %.inv27, i64 %2, i64 0
;  @ /home/pablo/Teaching/ensae/mie37/tutorials/1_Julia_Basics.ipynb:4 within `stupid_loop`
  %3 = select i1 %.inv26, i1 %.inv27, i1 false
  br i1 %3, label %L33.preheader.split.us.us.us, label %L85

L74.us.us:                                        ; preds = %L63.us.us.us
;  @ /home/pablo/Teaching/ensae/mie37/tutorials/1_Julia_Basics.ipynb:9 within `stupid_loop`
; ┌ @ range.jl:891 within `iterate`
; │┌ @ 

In [4]:
@code_native stupid_loop(10,10,10)

	.text
	.file	"stupid_loop"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3                               # -- Begin function julia_stupid_loop_1283
.LCPI0_0:
	.quad	0x3ff0000000000000              # double 1
	.text
	.globl	julia_stupid_loop_1283
	.p2align	4, 0x90
	.type	julia_stupid_loop_1283,@function
julia_stupid_loop_1283:                 # @julia_stupid_loop_1283
; ┌ @ /home/pablo/Teaching/ensae/mie37/tutorials/1_Julia_Basics.ipynb:1 within `stupid_loop`
	.cfi_startproc
# %bb.0:                                # %top
; │ @ /home/pablo/Teaching/ensae/mie37/tutorials/1_Julia_Basics.ipynb:3 within `stupid_loop`
; │┌ @ range.jl:5 within `Colon`
; ││┌ @ range.jl:397 within `UnitRange`
; │││┌ @ range.jl:404 within `unitrange_last`
	testq	%rdi, %rdi
	vxorpd	%xmm0, %xmm0, %xmm0
; │└└└
	jle	.LBB0_9
# %bb.1:                                # %L16.preheader
	testq	%rsi, %rsi
; │ @ /home/pablo/Teaching/ensae/mie37/tutorials/1_Julia_Basics.ipynb:4 within `stupid_loop`
	jle	.LBB0_9
# %bb.2:  

### Syntax Review

#### Variable assignment

Assignement operator is = (equality is ==, identity is ===)

In [4]:
# Assign the value 10 to the variable x
x = 10

10

In [5]:
x

10

In [8]:
4 == x

false

In [9]:
# Variable names can have Unicode characters
# To get ϵ in the REPL, type \epsilon<TAB>
a = 20

α = 10

# prey / predator 
🐳 = 0.1
🦈 = 0.1 * 🐳

σ = 34
ϵ = 1e-4

0.0001

Default semantic is pass-by-reference:

In [11]:
a = [1,2,3,4]
b = a
a[1] = 10
b

4-element Vector{Int64}:
 10
  2
  3
  4

To work on a copy: `copy` or `deepcopy`

In [13]:
a = [1,2,3,4]
b = copy(a)
a[1]=10
b

4-element Vector{Int64}:
 1
 2
 3
 4

In [15]:
a .== b 

4-element BitVector:
 0
 1
 1
 1

In [16]:
a .== b

4-element BitVector:
 1
 1
 1
 1

In [19]:
c === b

false

#### Basic types

In [16]:
# for any object `typeof` returns the type
typeof(a)

Vector{Int64} (alias for Array{Int64, 1})

In [15]:
[1,2,3]

3-element Vector{Int64}:
 1
 2
 3

In [19]:
typeof(randn(3,3)) == Matrix{Float64}

true

#### Numbers

In [22]:
y = 2 + 2

4

In [23]:
-y

-4

In [24]:
0.34*23

7.82

In [26]:
3 / 4

0.75

In [26]:
3 // 4 + 1//2

5//4

In [29]:
3//4 + 2//3

17//12

In [ ]:
typeof(3//4 + 2//3)

Rational{Int64}

In [29]:
# Scalar multiplication doesn't require *
3(4 - 2) 

12

In [30]:
x = 4
2x + 2x^2

40

In [38]:
typeof(big(100))

BigInt

In [41]:
factorial(big(1000))

4023872600770937735437024339230039857193748642107146325437999104299385123986290205920442084869694048004799886101971960586316668729948085589013238296699445909974245040870737599188236277271887325197795059509952761208749754624970436014182780946464962910563938874378864873371191810458257836478499770124766328898359557354325131853239584630755574091142624174743493475534286465766116677973966688202912073791438537195882498081268678383745597317461360853795345242215865932019280908782973084313928444032812315586110369768013573042161687476096758713483120254785893207671691324484262361314125087802080002616831510273418279777047846358681701643650241536913982812648102130927612448963599287051149649754199093422215668325720808213331861168115536158365469840467089756029009505376164758477284218896796462449451607653534081989013854424879849599533191017233555566021394503997362807501378376153071277619268490343526252000158885351473316117021039681759215109077880193931781141945452572238655414610628921879602238389714760

In [36]:
sizeof(x)

8

Int64

In [45]:
# (big(100))//big(1000)

1//10

In [44]:
bitstring(1)

"0000000000000000000000000000000000000000000000000000000000000001"

#### Booleans

Equality

In [45]:
0 == 1

false

In [46]:
2 != 3

true

In [51]:
3 < 4

true

In [47]:
true == false

false

Identity

In [ ]:
a = [34, 35]
b = [34, 35]
c = a

In [ ]:
c === a

In [ ]:
b === a

Boolean operator

In [54]:
true && false

false

In [55]:
true || false

true

In [56]:
!true

false

In [50]:
factorial(1000)

OverflowError: OverflowError: 1000 is too large to look up in the table; consider using `factorial(big(1000))` instead

In [52]:
a  = 2
b = 3

(a > b) && (factorial(1000) > 10)


false

#### Strings

In [53]:
# Strings are written using double quotes
str = "This is a string"

"This is a string"

In [54]:
ch = '🦆' # this is a character

'🦆': Unicode U+1F986 (category So: Symbol, other)

In [55]:
# Strings can also contain Unicode characters
fancy_str = "α is a string"

"α is a string"

In [64]:
n = 10
println("Iteration : ", n)

Iteration : 10

In [56]:
# String interpolation using $
# The expression in parentheses is evaluated and the result is 
# inserted into the string
a = 2+2
"2 + 2 = $(a+1)"

"2 + 2 = 5"

In [59]:
println("It took me $(a) iterations")

It took me 4 iterations


In [63]:
# String concatenation using *
"hello" * "world"

"helloworld"

In [60]:
print("1")
print("2")
print("3")

123

In [70]:
println("1")
println("2")
println("3")

1
2
3


In [44]:
println("hello ", "world")

hello world


#### Arrays

Julia has one-dimensional arrays. They are also called Vector.

In [64]:
A = [1, 2]

2-element Vector{Int64}:
 1
 2

All elements have the type:

In [66]:
A = [5, 1.4]

2-element Vector{Float64}:
 5.0
 1.4

In [73]:
typeof(A) == Vector{Int64}

false

In [69]:
A''

2-element Vector{Float64}:
 5.0
 1.4

To get the size of an array:

In [77]:
length(A)

2

In [70]:
size(A)

(2,)

Arrays are *mutable*

In [71]:
A[1] = 10

10

In [72]:
A

2-element Vector{Float64}:
 10.0
  1.4

Julia has one-based indexing: you refer to the first element as 1 ($\neq$ zero-based indexing in C or Python)

In [75]:
A[2]

1.4

Arrays are mutable and their size can be changed too:

In [78]:
push!(A, 29)
A

5-element Vector{Float64}:
 10.0
  1.4
 29.0
 29.0
 29.0

In [79]:
prepend!(A, -1)

6-element Vector{Float64}:
 -1.0
 10.0
  1.4
 29.0
 29.0
 29.0

In [80]:
prepend!(A, 28)

5-element Vector{Float64}:
 28.0
  1.0
  1.4
 29.0
 29.0

Two comments:
- the `push!` operation is *fast*
- `!` is a julia convention to express the fact that `push!` mutates its first argument

In [81]:
["a", "b"]

2-element Vector{String}:
 "a"
 "b"

In [82]:
["a", 1]

2-element Vector{Any}:
  "a"
 1

#### tuples

In [80]:
size(A)  # is a tuple

(6,)

In [81]:
(5,)

(5,)

In [82]:
# you can create tuples with (,,,)
t = (1,2,3,4)


(1, 2, 3, 4)

In [83]:
t

(1, 2, 3, 4)

tuples differ from arrays in two ways:
- they are immutable
- they can contain non-homogenous objects

In [84]:
t[1]

1

In [85]:
t[1] = 2

MethodError: MethodError: no method matching setindex!(::NTuple{4, Int64}, ::Int64, ::Int64)
The function `setindex!` exists, but no method is defined for this combination of argument types.

In [91]:
typeof((1, "1", [1]))

Tuple{Int64, String, Vector{Int64}}

2d arrays are also called matrices... and can be used for matrix multiplications.

In [86]:
[3 4  ;   5 6]

2×2 Matrix{Int64}:
 3  4
 5  6

In [89]:
[ [[3, 4] ;[2, 2]];;; [[3, 4]; [5, 6]]] # concatenate along second dimension

4×1×2 Array{Int64, 3}:
[:, :, 1] =
 3
 4
 2
 2

[:, :, 2] =
 3
 4
 5
 6

In [ ]:
a1 = [1,2,3,4]
a2 = [1,2,3,4]  .+ 4
# [a1 ;; a2]
cat(a1, a2; dims=2)

4×2 Matrix{Int64}:
 1  5
 2  6
 3  7
 4  8

In [37]:
b = [1 0.6 0]

1×3 Array{Float64,2}:
 1.0  0.6  0.0

In [ ]:
B = [0.1 0.2 0.3; 4 5 6]

Other ways to construct arrays:

In [94]:
# zero array
t = zeros(2,3)
t[1,2] = 23.2
t

2×3 Matrix{Float64}:
 0.0  23.2  0.0
 0.0   0.0  0.0

In [97]:
# zero array
t = zeros(Int64,2,3)
t[1,2] = 23
t

2×3 Matrix{Int64}:
 0  23  0
 0   0  0

In [79]:
# random array (uniform distribution)
t= rand(3,3)
t

3×3 Matrix{Float64}:
 0.151296  0.390327  0.239194
 0.726286  0.371063  0.133779
 0.037311  0.183624  0.72499

In [99]:
# random array (normal distribution)
t= randn(3,3,2,2)
t

3×3×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 -1.87278  -0.309205   0.249804
 -1.93702  -0.331663   1.7964
  1.31126  -0.730084  -0.488771

[:, :, 2, 1] =
 -1.02687    1.27482   0.207381
  0.954285  -1.5966   -0.377572
 -0.725551  -1.47646   0.221127

[:, :, 1, 2] =
 -0.943978  -0.482072   -1.26438
 -1.01204    0.0370359   0.111284
 -0.361247  -0.0701506  -1.27103

[:, :, 2, 2] =
 -1.10099     0.387518   0.825336
 -1.12199    -0.975774  -1.24188
 -0.0136014   1.39104    0.579525

Vectorized operations take a ., even comparisons (pointwise operations)

In [102]:
B = [1 2;3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [103]:
B*B

2×2 Matrix{Int64}:
  7  10
 15  22

In [104]:
B .* B

2×2 Matrix{Int64}:
 1   4
 9  16

In [ ]:
f(x) = x^2 + 1

f (generic function with 1 method)

In [106]:
f(43)

1850

In [108]:
f.(B)

2×2 Matrix{Int64}:
  2   5
 10  17

In [108]:
# [ f(e) for e in [1,2,3,4,5] ]
f.([1,2,3,4,5])

5-element Vector{Int64}:
  2
  5
 10
 17
 26

Elements are always accessed with square brackets:

In [109]:
B = [1 2 3; 4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [ ]:
You get element $B_{ij}$ with `B[i,j]`

In [109]:
B[1,2]

2

You select a whole row/column with `:`

In [112]:
B = [B ;; [5,6]]

2×3 Matrix{Int64}:
 1  2  5
 3  4  6

In [113]:
B[1,:]

3-element Vector{Int64}:
 1
 2
 5

In [115]:
B

2×3 Matrix{Int64}:
 1  2  5
 3  4  6

In [114]:
B[:,1:2]

2×2 Matrix{Int64}:
 1  2
 3  4

In [117]:
B[:,1:end-1]

2×2 Matrix{Int64}:
 1  2
 3  4

#### Control flow

Conditions

In [97]:
x = 0
if x<0
    # block
    println("x is negative")
elseif (x > 0) # optional and unlimited
    println("x is positive")
else         # optional
    println("x is zero")
end

x is zero


While

In [119]:
i = 3
while i > 0
    println(i)
    i -= 1 # decrement
end

3
2
1


For loops: your iterate over any iterable object:
- range   `i1:i2`
- vector
- tuple

In [122]:
# Iterate through ranges of numbers
for i ∈ (1:3)
    println(i)
end

1
2
3


In [123]:
# Iterate through arrays
cities = ["Boston", "New York", "Philadelphia"]
for city ∈ cities
    println(city)
end

Boston
New York
Philadelphia


In [100]:
cities

3-element Vector{String}:
 "Boston"
 "New York"
 "Philadelphia"

In [124]:
states = ["Massachussets", "New York", "Pennsylvania"]

3-element Vector{String}:
 "Massachussets"
 "New York"
 "Pennsylvania"

In [127]:
for i = 1:length(cities)
    println(cities[i], " : ", states[i])
end

Boston : Massachussets
New York : New York
Philadelphia : Pennsylvania


In [131]:
for (i,c) in enumerate(cities)
    println(c, " : ", states[i])
end

Boston : Massachussets
New York : New York
Philadelphia : Pennsylvania


In [136]:
for (c,s) in zip(cities, states)
    println("state: $s \t city: $c")
end

state: Massachussets 	 city: Boston
state: New York 	 city: New York
state: Pennsylvania 	 city: Philadelphia


In [ ]:
for (i,c) in enumeratea(cities)
    println(c, " : ", states[i])
end

In [118]:
two_by_two_iterable = zip(cities, states)

zip(["Boston", "New York", "Philadelphia"], ["Massachussets", "New York", "Pennsylvania"])

In [120]:
typeof(two_by_two_iterable)

Base.Iterators.Zip{Tuple{Vector{String}, Vector{String}}}

In [121]:
collect(two_by_two_iterable)

3-element Vector{Tuple{String, String}}:
 ("Boston", "Massachussets")
 ("New York", "New York")
 ("Philadelphia", "Pennsylvania")

In [122]:
[two_by_two_iterable...]

3-element Vector{Tuple{String, String}}:
 ("Boston", "Massachussets")
 ("New York", "New York")
 ("Philadelphia", "Pennsylvania")

In [106]:
# Iterate through arrays of tuples using zip
for kw in zip(cities, states)
    println(kw)
end

("Boston", "Massachussets")
("New York", "New York")
("Philadelphia", "Pennsylvania")


In [123]:
# Iterate through arrays of tuples using zip
for (city, state) in zip(cities, states)
    println("City: $city | State: $state")
end

City: Boston | State: Massachussets
City: New York | State: New York
City: Philadelphia | State: Pennsylvania


In [108]:
# Iterate through arrays and their indices using enumerate
for (i, city) in enumerate(cities)
    println("City $i is $city")
end

City 1 is Boston
City 2 is New York
City 3 is Philadelphia


#### List comprehensions

In [111]:
[1:10 ...] # unpack operator

10-element Vector{Float64}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0

In [137]:
[i^2 for i  in 1:10] # collect with comprehension syntax

10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [125]:
[i^2 for i=1:10000000 if mod(i,2)==0] ;

In [133]:
@time sum( [i^2 for i=1:10000000000 if mod(i,2)==0] )

In [1]:
function fun()
    t = 0
    for  i=1:10000000000
        if mod(i,2)==0
            t += i^2
        end
    end
    return t
end

fun (generic function with 1 method)

In [2]:
@time fun()

  2.552283 seconds


-4494074839411110912

In [8]:
gen = (i^2 for i=1:10000000000 if mod(i,2)==0)

Base.Generator{Base.Iterators.Filter{var"#24#26", UnitRange{Int64}}, var"#23#25"}(var"#23#25"(), Base.Iterators.Filter{var"#24#26", UnitRange{Int64}}(var"#24#26"(), 1:10000000000))

In [10]:
@time sum(gen)

 16.349360 seconds (1 allocation: 16 bytes)


-4494074839411110912

In [11]:
## Named Tuples

In [13]:
t = (;a=1,b=2,c=3)

(a = 1, b = 2, c = 3)

In [17]:
t[1] # indexed like tuple
# t[1] = 2 # immutable
t.a # access fields using names

1

In [18]:
model = (;
    α = 0.3,
    β = 0.96
)

(α = 0.3, β = 0.96)

In [21]:
merge(model, (;β=0.9, γ=0.2))

(α = 0.3, β = 0.9, γ = 0.2)

In [23]:
# unpack values from a tuple

α = model[1]
β = model[2]

0.96

In [24]:
# unpack values from a namedtuple

α = model.α
β = model.β

0.96

In [27]:
# namedtuple unpacking

(;α, β) = model
α

0.3

0.3

### Data Types and multiple dispatch

#### Composite types

A **composite type** is a collection of named fields that can be treated as a single value. They bear a passing resemblance to MATLAB structs.

All fields must be declared ahead of time. The double colon, `::`, constrains a field to contain values of a certain type. This is optional for any field.

In [98]:
# Type definition with 4 fields
struct ParameterFree
    value  
    transformation  
    tex_label
    description 
end

In [95]:
pf = ParameterFree("1", x->x^2, "\\sqrt{1+x^2}", ("a",1))

ParameterFree("1", var"#9#10"(), "\\sqrt{1+x^2}", ("a", 1))

In [99]:
pf.value

"1"

Two reasons to create structures:
- syntactic shortcut (you access the fields with .)
- specify the types of the fields

In [100]:
# Type definition
struct Parameter
    value ::Float64
    transformation ::Function # Function is a type!
    tex_label::String
    description::String
end

In [101]:
p = Parameter("1", x->x^2, "\\sqrt{1+x^2}", ("a",1))

LoadError: [91mMethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mFloat64[39m[39m
[91m[0mClosest candidates are:[39m
[91m[0m  convert(::Type{T}, [91m::T[39m) where T<:Number at number.jl:6[39m
[91m[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number at number.jl:7[39m
[91m[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at twiceprecision.jl:250[39m
[91m[0m  ...[39m

In [102]:
p = Parameter(0.43, x->x^2, "\\sqrt{1+x^2}", "This is a description")

Parameter(0.43, var"#13#14"(), "\\sqrt{1+x^2}", "This is a description")

In [103]:
p.value

0.43

When a type with $n$ fields is defined, a constructor (function that creates an instance of that type) that takes $n$ ordered arguments is automatically created. Additional constructors can be defined for convenience.

In [104]:
# Creating an instance of the Parameter type using the default
# constructor
β = Parameter(0.9, identity, "\\beta", "Discount rate")

Parameter(0.9, identity, "\\beta", "Discount rate")

In [106]:
function Parameter(value)
    return Parameter(value, x->x, "x", "Anonymous")
end

Parameter

In [107]:
Parameter(0.4)

Parameter(0.4, var"#15#16"(), "x", "Anonymous")

In [108]:
Parameter(value, transformation, tex) = Parameter(value, transformation, tex, "no description")

Parameter

In [109]:
methods( Parameter )

# 4 methods for type constructor:
[1] Parameter(value::Float64, transformation::Function, tex_label::String, description::String) in Main at In[100]:3
[2] Parameter(value) in Main at In[106]:1
[3] Parameter(value, transformation, tex) in Main at In[108]:1
[4] Parameter(value, transformation, tex_label, description) in Main at In[100]:3

In [110]:
# Alternative constructors end with an appeal to the default
# constructor
function Parameter(value::Float64, tex_label::String)
    transformation = identity
    description = "No description available"
    return Parameter(value, transformation, tex_label, description)
end

α = Parameter(0.5, "\alpha")

Parameter(0.5, identity, "\alpha", "No description available")

Now the function `Parameter` has two different `methods` with different signatures:

In [9]:
methods(Parameter)

# 4 methods for type constructor:
[1] Parameter(value::Float64, transformation::Function, tex_label::String, description::String) in Main at In[1]:3
[2] Parameter(value::Float64, tex_label::String) in Main at In[8]:4
[3] Parameter(value, transformation, tex) in Main at In[5]:1
[4] Parameter(value, transformation, tex_label, description) in Main at In[1]:3

We have seen that a function can have several implementations, called methods, for different number of arguments, 
or for different types of arguments.

In [111]:
fun(x::Int64, y::Int64) = x^3 + y

fun (generic function with 1 method)

In [114]:
fun(x::Float64, y::Int64) = x/2 + y

fun (generic function with 2 methods)

In [115]:
fun(2, 2)

10

In [117]:
fun(2.0, 2)

3.0

In [ ]:
α.tex_label

In [118]:
# Access a particular field using .
α.value

0.5

In [ ]:
# Fields are modifiable and can be assigned to, like 
# ordinary variables
α.value = 0.75

### Mutable vs non mutable types


by default structures in Julia are non-mutable

In [123]:
p.value = 3.0

LoadError: [91msetfield! immutable struct of type Parameter cannot be changed[39m

In [124]:
mutable struct Params
    x:: Float64
    y:: Float64
end

In [125]:
pos = Params(0.4, 0.2)

Params(0.4, 0.2)

In [126]:
pos.x = 0.5

0.5

### Parameterized Types

**Parameterized types** are data types that are defined to handle values identically regardless of the type of those values.

Arrays are a familiar example. An `Array{T,1}` is a one-dimensional array filled with objects of any type `T` (e.g. `Float64`, `String`).

In [127]:
# Defining a parametric point
struct Duple{T} # T is a parameter to the type Duple
    x::T
    y::T
end

In [130]:
Duple(3, 3)

Duple{Int64}(3, 3)

In [128]:
Duple(3, -1.0)

LoadError: [91mMethodError: no method matching Duple(::Int64, ::Float64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  Duple(::T, [91m::T[39m) where T at In[127]:3[39m

In [29]:
struct Truple{T}
    x::Duple{T}
    z::T
end

This single declaration defines an unlimited number of new types: `Duple{String}`, `Duple{Float64}`, etc. are all immediately usable.

In [30]:
sizeof(3.0)

8

In [31]:
sizeof( Duple(3.0, -15.0) )

16

In [ ]:
# What happens here?
Duple(1.5, 3)

In [35]:
struct Truple3{T,S}
    x::Tuple{T,S}
    z::S
end

We can also restrict the type parameter `T`:

In [37]:
typeof("S") <: Number

false

In [38]:
typeof(4) <: Number

true

In [40]:
# T can be any subtype of Number, but nothing else
struct PlanarCoordinate{T<:Number}
    x::T
    y::T
end

In [41]:
PlanarCoordinate("4th Ave", "14th St")

MethodError: MethodError: no method matching PlanarCoordinate(::String, ::String)

In [42]:
PlanarCoordinate(2//3, 8//9)

PlanarCoordinate{Rational{Int64}}(2//3, 8//9)

Arrays are an exemple of mutable, parameterized types

### Why Use Types?

You can write all your code without thinking about types at all. If you do this, however, you’ll be missing out on some of the biggest benefits of using Julia.

If you understand types, you can:

- Write faster code
- Write expressive, clear, and well-structured programs (keep this in mind when we talk about functions)
- Reason more clearly about how your code works

Even if you only use built-in functions and types, your code still takes advantage of Julia’s type system. That’s why it’s important to understand what types are and how to use them.

In [49]:
# Example: writing type-stable functions
function sumofsins_unstable(n::Integer)  
    sum = 0:: Integer
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end  

function sumofsins_stable(n::Integer)  
    sum = 0.0 :: Float64
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end

# Compile and run
sumofsins_unstable(Int(1e5))
sumofsins_stable(Int(1e5))

-25554.110202663698

In [50]:
@time sumofsins_unstable(Int(1e5))

  0.000176 seconds


-25554.110202663698

In [47]:
@time sumofsins_stable(Int(1e5))

  0.000168 seconds


-25554.110202663698

In `sumofsins_stable`, the compiler is guaranteed that `sum` is of type `Float64` throughout; therefore, it saves time and memory. On the other hand, in `sumofsins_unstable`, the compiler must check the type of `sum` at each iteration of the loop. Let's look at the LLVM [intermediate representation](http://www.johnmyleswhite.com/notebook/2013/12/06/writing-type-stable-code-in-julia/).

### Multiple Dispatch

So far we have defined functions over argument lists of any type. Methods allow us to define functions “piecewise”. For any set of input arguments, we can define a **method**, a definition of one possible behavior for a function.

In [51]:
# Define one method of the function print_type
function print_type(x::Number)
    println("$x is a number")
end

print_type (generic function with 1 method)

In [53]:
# Define another method
function print_type(x::String)
    println("$x is a string")
end

print_type (generic function with 2 methods)

In [54]:
# Define yet another method
function print_type(x::Number, y::Number)
    println("$x and $y are both numbers")
end

print_type (generic function with 3 methods)

In [55]:
# See all methods for a given function
methods(print_type)

# 3 methods for generic function "print_type":
[1] print_type(x::String) in Main at In[53]:3
[2] print_type(x::Number) in Main at In[51]:3
[3] print_type(x::Number, y::Number) in Main at In[54]:3

Julia uses **multiple dispatch** to decide which **method** of a **function** to execute when a function is applied. In particular, Julia compares the types of _all_ arguments to the signatures of the function’s methods in order to choose the applicable one, not just the first (hence "multiple").

In [56]:
print_type(5)

5 is a number


In [57]:
print_type("foo")

foo is a string


In [58]:
print_type([1, 2, 3])

MethodError: MethodError: no method matching print_type(::Array{Int64,1})
Closest candidates are:
  print_type(!Matched::String) at In[53]:3
  print_type(!Matched::Number) at In[51]:3
  print_type(!Matched::Number, !Matched::Number) at In[54]:3

#### Other types of functions

Julia supports a short function definition for one-liners

In [ ]:
f(x::Float64) = x^2.0
f(x::Int64) = x^3

As well as a special syntax for anonymous functions

In [ ]:
u->u^2

In [ ]:
map(u->u^2, [1,2,3,4])

### Keyword arguments and optional arguments

In [59]:
f(a,b,c=true; algo="newton")

UndefVarError: UndefVarError: f not defined

### Packing/unpacking


In [60]:
t = (1,2,4)

(1, 2, 4)

In [61]:
a,b,c = t

(1, 2, 4)

In [65]:
[(1:10)...]

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [66]:
cat([4,3], [0,1]; dims=1)

4-element Array{Int64,1}:
 4
 3
 0
 1

In [67]:
l = [[4,3], [0,1], [0, 0], [1, 1]]
# how do I concatenate it ?

cat(l...; dims=1) ### see python's f(*s)

8-element Array{Int64,1}:
 4
 3
 0
 1
 0
 0
 1
 1

### Writing Julian Code

As we've seen, you can use Julia just like you use MATLAB and get faster code. However, to write faster and _better_ code, attempt to write in a “Julian” manner:

- Define composite types as logically needed
- Write type-stable functions for best performance
- Take advantage of multiple dispatch to write code that looks like math
- Add methods to existing functions

### Just-in-Time Compilation

How is Julia so fast? Julia is just-in-time (JIT) compiled, which  means (according to [this StackExchange answer](http://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do)):

> A JIT compiler runs after the program has started and compiles the code (usually bytecode or some kind of VM instructions) on the fly (or just-in-time, as it's called) into a form that's usually faster, typically the host CPU's native instruction set. _A JIT has access to dynamic runtime information whereas a standard compiler doesn't and can make better optimizations like inlining functions that are used frequently._

> This is in contrast to a traditional compiler that compiles all the code to machine language before the program is first run.

In particular, Julia uses type information at runtime to optimize how your code is compiled. This is why writing type-stable code makes such a difference in speed!

## Additional Exercises

Taken from QuantEcon's [Julia Essentials](https://lectures.quantecon.org/jl/julia_essentials.html) and [Vectors, Arrays, and Matrices](https://lectures.quantecon.org/jl/julia_arrays.html) lectures.

1. Consider the polynomial $$p(x) = \sum_{i=0}^n a_0 x^0$$ Using `enumerate`, write a function `p` such that `p(x, coeff)` computes the value of the polynomial with coefficients `coeff` evaluated at `x`.

ppp (generic function with 1 method)

2. Write a function `solve_discrete_lyapunov` that solves the discrete Lyapunov equation $$S = ASA' + \Sigma \Sigma'$$ using the iterative procedure $$S_0 = \Sigma \Sigma'$$ $$S_{t+1} = A S_t A' + \Sigma \Sigma'$$ taking in as arguments the $n \times n$ matrix $A$, the $n \times k$ matrix $\Sigma$, and a number of iterations.